In [1]:
import os
import numpy as np
import pandas as pd
import json
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table
from jupyter_dash import JupyterDash
from wordcloud import WordCloud, STOPWORDS
from nltk.stem.porter import *

In [3]:
users = []
dir = os.getcwd() # gets the current work directory
files  = os.listdir(dir) # gets all the file names into files variable
for file in files: # reading all the csv files into a list
    if file.endswith(".csv"):
        users.append(pd.read_csv(file)) 

In [3]:
i = 0
listing = []
while i < len(users):
    use = users[i].dropna()
    response_data = use.response_content
    dff = []
    for dictionary_string in response_data:
        try:
            dictionary = json.loads(dictionary_string)
        except:
            print(dictionary_string)
        dff.append(dictionary)
    df = pd.DataFrame(dff)
    listing.append(df)
    i=i +1

response_content
response_content
response_content


In [4]:
merged = pd.concat(listing) # This merges the list of dataframes into one dataframe

In [5]:
merged.reset_index(drop=True, inplace=True)

In [6]:
merged.head()

,created_at,id,id_str,text,truncated,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,possibly_sensitive,lang,extended_entities,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status,withheld_scope,withheld_copyright,withheld_in_countries
0,Fri May 01 22:45:54 +0000 2020,1256354186309951489,1256354186309951489,Thoughtful presents for mom https://t.co/cYWva...,False,"{'hashtags': [{'text': 'MothersDay', 'indices'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,...,False,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fri May 01 19:18:50 +0000 2020,1256302075656323072,1256302075656323072,@lildickytweets @DaveOnFXX https://t.co/zT5Hpj...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,1.20952e+09,...,False,und,"{'media': [{'id': 1256301968290508800, 'id_str...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fri May 01 16:15:22 +0000 2020,1256255906129408000,1256255906129408000,New items added to my closet at https://t.co/w...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,...,False,en,"{'media': [{'id': 1256255867973857280, 'id_str...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fri May 01 02:53:33 +0000 2020,1256054122752557057,1256054122752557057,You won’t see this on tonight’s episode: https...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,...,False,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fri May 01 00:07:21 +0000 2020,1256012297501212673,1256012297501212673,RT @waveskylie: #KUWTK TIME @khloekardashian @...,False,"{'hashtags': [{'text': 'KUWTK', 'indices': [16...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,...,False,en,"{'media': [{'id': 1253487032568909824, 'id_str...",{'created_at': 'Fri May 01 00:00:45 +0000 2020...,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
all_data = pd.concat([merged.drop(['user'], axis=1), merged['user'].apply(pd.Series)], axis=1) # This code unstacks the user column

In [8]:
all_data.columns

Index(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'retweet_count', 'favorite_count',
       'favorited', 'retweeted', 'possibly_sensitive', 'lang',
       'extended_entities', 'retweeted_status', 'quoted_status_id',
       'quoted_status_id_str', 'quoted_status', 'withheld_scope',
       'withheld_copyright', 'withheld_in_countries', 'id', 'id_str', 'name',
       'screen_name', 'location', 'description', 'url', 'entities',
       'protected', 'followers_count', 'friends_count', 'listed_count',
       'created_at', 'favourites_count', 'utc_offset', 'time_zone',
       'geo_enabled', 'verified', 'statuses_count', 'lang',
       'contributors_enabled', 'is_translator', 'is_translation_enabled',
       'profile_background_color', 'profile

In [9]:
columns = ['name', 'retweet_count', 'favorite_count', 'followers_count', 'friends_count','listed_count','created_at','statuses_count', 'text']

In [10]:
df= all_data[columns]

In [11]:
df.columns= ['name', 'retweet_count', 'favorite_count',  'followers_count', 'friends_count','listed_count', 'tweet_date', 'created_at','statuses_count', 'text']

In [12]:
df.loc[:,'tweet_date'] =pd.to_datetime(df.loc[:,'tweet_date']).dt.date

/home/pytha_goras/snap/jupyter/common/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [13]:
df.loc[:,'created_at'] =pd.to_datetime(df.loc[:,'created_at']).dt.date

In [14]:
df.loc[:,'year'] = pd.to_datetime(df.loc[:,'tweet_date']).dt.year

/home/pytha_goras/snap/jupyter/common/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [15]:
df.head()

,name,retweet_count,favorite_count,followers_count,friends_count,listed_count,tweet_date,created_at,statuses_count,text,year
0,Kourtney Kardashian,17,373,25030622,75,29530,2020-05-01,2009-03-10,13509,Thoughtful presents for mom https://t.co/cYWva...,2020
1,Kourtney Kardashian,131,1492,25030622,75,29530,2020-05-01,2009-03-10,13509,@lildickytweets @DaveOnFXX https://t.co/zT5Hpj...,2020
2,Kourtney Kardashian,26,548,25030622,75,29530,2020-05-01,2009-03-10,13509,New items added to my closet at https://t.co/w...,2020
3,Kourtney Kardashian,49,1194,25030622,75,29530,2020-05-01,2009-03-10,13509,You won’t see this on tonight’s episode: https...,2020
4,Kourtney Kardashian,21,0,25030622,75,29530,2020-05-01,2009-03-10,13509,RT @waveskylie: #KUWTK TIME @khloekardashian @...,2020


In [16]:
df_user = df[['name','followers_count', 'listed_count','created_at', 'statuses_count']]

In [21]:
df_user = df_user.drop_duplicates(subset='name', keep='first').reset_index(drop=True)

In [22]:
df_user.loc[:,'age_in_months'] = (dt.date.today()- df_user.loc[:,'created_at']).dt.days/30

In [23]:
df_user

,name,followers_count,listed_count,created_at,statuses_count,age_in_months
0,Kourtney Kardashian,25030622,29530,2009-03-10,13509,138.000000
1,Mesut Özil,24891702,15744,2012-03-22,1755,101.066667
2,el-p,235409,2022,2009-02-15,1091,138.766667
3,NASA,36407272,95670,2007-12-19,60112,152.900000
4,Donald J. Trump,78392620,120142,2009-03-18,50927,137.733333
5,CNN Breaking News,57537648,186817,2007-01-02,70305,164.600000
6,Drizzy,39076866,40441,2009-03-28,1752,137.400000
7,Neymar Jr,46776185,50372,2010-06-22,41652,122.366667
8,Chris Brown,31820517,44578,2010-03-03,4549,126.066667
9,Kim Kardashian West,64606721,80830,2009-03-19,32193,137.700000


In [24]:
#### Finding the tweets per days
df_user.loc[:,'tweets_per_month']=df_user.loc[:,'statuses_count']//(df_user.loc[:,'age_in_months'])

In [25]:
# Listings per followers. I call it importance to followers
df_user.loc[:,'importance']= df_user.loc[:,'listed_count']/(df_user.loc[:,'followers_count'])

In [27]:
df_user

,name,followers_count,listed_count,created_at,statuses_count,age_in_months,tweets_per_month,importance
0,Kourtney Kardashian,25030622,29530,2009-03-10,13509,138.000000,97.0,0.001180
1,Mesut Özil,24891702,15744,2012-03-22,1755,101.066667,17.0,0.000632
2,el-p,235409,2022,2009-02-15,1091,138.766667,7.0,0.008589
3,NASA,36407272,95670,2007-12-19,60112,152.900000,393.0,0.002628
4,Donald J. Trump,78392620,120142,2009-03-18,50927,137.733333,369.0,0.001533
5,CNN Breaking News,57537648,186817,2007-01-02,70305,164.600000,427.0,0.003247
6,Drizzy,39076866,40441,2009-03-28,1752,137.400000,12.0,0.001035
7,Neymar Jr,46776185,50372,2010-06-22,41652,122.366667,340.0,0.001077
8,Chris Brown,31820517,44578,2010-03-03,4549,126.066667,36.0,0.001401
9,Kim Kardashian West,64606721,80830,2009-03-19,32193,137.700000,233.0,0.001251


In [38]:

blue={
    'height':'120px',
    'background-color':'blue',
    'width':'50%',
    'display':'inline-block'}
green={
       'height':'120px',
       'background-color':'brown',
       'width':'50%',
       'display':'inline-block'}



app = JupyterDash(__name__)

names = df.name.unique()

app.layout = html.Div([
    html.H1(children="Tweet Dashboard", 
             style={
                 'text-align':'center',
                 'background-color': 'lightblue'}),
    
    html.Div(children = "General Statistics of users",
            style={
                "text-align":"center"
            }),
    dcc.Graph(id="most_active", figure={},
             style= {'width': '49%', 'display': 'inline-block'}),
    dcc.Graph(id="oldest_members", figure={},
             style= {'width': '49%', 'display': 'inline-block'}),
    dcc.Graph(id="importance", figure={},
             style= {'width': '49%', 'display': 'inline-block'}),
    dcc.Graph(id="age_follow", figure={},
             style= {'width': '49%', 'display': 'inline-block'}),
    
    
    html.H1(children="Statistics on Users", 
             style={
                 'text-align':'center',
                 'background-color': 'lightblue'}),
    
    dcc.Dropdown(id="tweet_name", 
                options=[{'label': i, 'value': i} for i in names],
                placeholder='Select a user',
                style={'width':'50%'},
                value= "NASA"
               
           ),
    
    html.Br(),
    dcc.Graph(id="retweet_follow", figure={},
             style= {'width': '49%', 'display': 'inline-block'}),
    
    dcc.Graph(id="yearly_tweets", figure={},
             style= {'width': '49%', 'display': 'inline-block'}),
    
    dcc.Graph(id="retweet_trends", figure={},
             style= {'width': '100%', 'display': 'inline-block'}),
    
])
# Connecting the Plotly graphs with Dash Components
@app.callback(
        [Output(component_id='most_active', component_property="figure"),
         Output(component_id='oldest_members', component_property="figure"),
         Output(component_id='importance', component_property="figure"),
         Output(component_id='age_follow', component_property="figure"),
         Output(component_id='retweet_follow', component_property="figure"),
         Output(component_id='yearly_tweets', component_property="figure"),
         Output(component_id='retweet_trends', component_property="figure"),
        ],
         [Input(component_id='tweet_name', component_property='value')]
)

def update_graph(option_slct):
    #print(tweet_names)
    #print(type(tweet_names))
    #container = "Statistics on {}".format(tweet_name)
    dff = df.copy()
    dff = dff[dff['name']==option_slct]
    
    # The graph for the 20 most active members
    most_active = df_user.sort_values('tweets_per_month',ascending=False).reset_index(drop=True)
    fig1 = px.bar(most_active.loc[0:20, :], x='tweets_per_month', y="name", orientation = 'h', title='20 MOST ACTIVE MEMBERS', hover_data=['name','followers_count'])
    
    #Correlation between the followers and how long they have been on twitter
    fig2 = px.scatter(df_user, x = 'age_in_months', y="followers_count", title='FOLLOWERS VRS AGE',hover_data=['name'])
    
    #Graph for 20 oldest members
    age = df_user.sort_values('age_in_months',ascending=False).reset_index(drop=True).loc[0:20, :]
    fig3 = px.bar(age, x = 'statuses_count', y="name", title='NUMBER OF TWEETS FOR THE OLDEST 20', hover_data=['age_in_months'])
    
    # Listing to follower ratio. Top 20
    important = df_user.sort_values('importance',ascending=False).reset_index(drop=True).iloc[0:20, :]
    fig4 = px.bar(important, x = 'name', y="importance", orientation='v', title='LISTINGS/FOLOWERS', hover_data=['listed_count', 'followers_count'])
    
    # Retweet Trend
    fig7 = px.line(dff, x='tweet_date', y='retweet_count', hover_data=['text'], title = 'THE TREND OF RETWEETS')

    #yearly tweets
    status = dff.groupby(['year']).count().reset_index()
    fig6 = px.bar(status, x='year', y='statuses_count',title = 'NUMBER OF TWEETS IN A YEAR', hover_data=['text'])
    fig6.update_layout(clickmode='event+select')
    
    # Correlation between favorite and retweet
    fig5 = px.scatter(dff, x='favorite_count', y='retweet_count', hover_data=['text'], title='RETWEET COUNT VRS FAVORITE COUNT PER TWEET')
    
    
    
    
    return fig1, fig2, fig3, fig4, fig5, fig6, fig7
    
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [34]:
most_active = df_user.sort_values('tweets_per_month',ascending=False).reset_index(drop=True)
most_active

,name,followers_count,listed_count,created_at,statuses_count,age_in_months,tweets_per_month,importance
0,Xbox,14025089,18177,2009-03-16,260534,137.800000,1890.0,0.001296
1,CNN,47621430,145751,2007-02-09,297486,163.333333,1821.0,0.003061
2,FC Barcelona (from 🏠),32497048,32055,2009-12-15,108394,128.666667,842.0,0.000986
3,Chelsea FC - #StayHomeSaveLives,14301234,23994,2009-03-05,94790,138.166667,686.0,0.001678
4,UEFA Champions League,28351563,22118,2012-07-05,65678,97.566667,673.0,0.000780
5,Manchester United,21836403,22649,2012-04-20,58919,100.100000,588.0,0.001037
6,CNN Breaking News,57537648,186817,2007-01-02,70305,164.600000,427.0,0.003247
7,NASA,36407272,95670,2007-12-19,60112,152.900000,393.0,0.002628
8,Chance The Rapper,8326980,8420,2010-08-22,46636,120.333333,387.0,0.001011
9,Donald J. Trump,78392620,120142,2009-03-18,50927,137.733333,369.0,0.001533


In [69]:
age = df_user.sort_values('age_in_months',ascending=True).reset_index(drop=True).loc[0:20, :]
fig3 = px.bar(age, x = 'statuses_count', y="name")
fig3.show()

In [59]:
age

,name,followers_count,listed_count,created_at,statuses_count,age_in_months,tweets_per_month,importance
0,Kourtney Kardashian,25030622,29530,2009-03-10,13509,138.000000,97.0,0.001180
14,ALICE GLASS,63885,365,2009-03-10,719,138.000000,5.0,0.005713
18,LeBron James,45898283,46087,2009-03-06,6965,138.133333,50.0,0.001004
25,Chelsea FC - #StayHomeSaveLives,14301234,23994,2009-03-05,94790,138.166667,686.0,0.001678
23,Mike Pence,4636971,9383,2009-02-27,8608,138.366667,62.0,0.002024
16,Coldplay🌙☀️,23612844,45640,2009-01-11,7828,139.933333,55.0,0.001933
15,Narendra Modi,55791334,25880,2009-01-10,26562,139.966667,189.0,0.000464
20,Britney Spears,56165349,116183,2008-09-22,5639,143.633333,39.0,0.002069


In [66]:
df_user.sort_values('age_in_months', ascending=True).reset_index(drop=True).loc[0:20, :]

,name,followers_count,listed_count,created_at,statuses_count,age_in_months,tweets_per_month,importance
0,Kim Gordon,139016,1312,2012-08-07,9108,96.466667,94.0,0.009438
1,UEFA Champions League,28351563,22118,2012-07-05,65678,97.566667,673.0,0.000780
2,Manchester United,21836403,22649,2012-04-20,58919,100.100000,588.0,0.001037
3,Mesut Özil,24891702,15744,2012-03-22,1755,101.066667,17.0,0.000632
4,Lorde,7495817,8372,2011-08-15,2,108.400000,0.0,0.001117
5,방탄소년단,25346490,69453,2011-07-14,12253,109.466667,111.0,0.002740
6,Kylie Jenner,32763389,15182,2011-01-11,13868,115.600000,119.0,0.000463
7,Adele,27257640,31707,2010-08-30,310,120.066667,2.0,0.001163
8,Chance The Rapper,8326980,8420,2010-08-22,46636,120.333333,387.0,0.001011
9,Liam,32955614,97587,2010-06-22,11219,122.366667,91.0,0.002961


In [32]:
px.scatter?

In [33]:
px.bar?

In [36]:
important = df_user.sort_values('importance',ascending=False).reset_index(drop=True).iloc[0:20, :]
important.head()

,name,followers_count,listed_count,created_at,statuses_count,age_in_months,tweets_per_month,importance
0,The Mountain Goats,181475,2118,2009-08-21,37609,132.533333,283.0,0.011671
1,Kim Gordon,139016,1312,2012-08-07,9108,96.466667,94.0,0.009438
2,el-p,235409,2022,2009-02-15,1091,138.766667,7.0,0.008589
3,ALICE GLASS,63885,365,2009-03-10,719,138.000000,5.0,0.005713
4,CNN Breaking News,57537648,186817,2007-01-02,70305,164.600000,427.0,0.003247
